# PURPOSE:

### To obtain the top 3 and bottom 3 GraphCat adjustments in USD

**Procedure:** User enters exchange rate, copy their graphcat data into "clipboard", and then execute all cells in this notebook

In [1]:
from pathlib import Path
import os
import pandas as pd
import numpy as np
import scrapbook as sb
from win10toast import ToastNotifier
pd.options.display.max_rows=1000
pd.options.display.max_columns=100
pd.options.display.float_format = '{:20,.2f}'.format

In [2]:
#exchange_rate = float(input("Enter exchange rate: "))

Enter exchange rate: 0.7464


### Parameters that will be used by the papermill library:

In [3]:
exchange_rate = 0.7464
start_year = 2010
end_year = 2019
CLA_MONTH = '201901'

In [4]:
# df = pd.read_clipboard()

In [4]:
nb = sb.read_notebook('D:\\jupyter\\rvms\\production\\output\\RedAndGreenSheet.ipynb')

In [5]:
nb.scraps['path_to_red_green_sheet_excel_file'].data

'\\\\207.130.185.67\\aqgbudget2\\Cost\\Reserve Adjustments\\Reports\\Normal Reserve Balance Verification\\RVMS_Before_After_Checks\\201901\\All_Plants_Before_After_Budgeted_CPUs_2019-03-21_13_06.xlsx'

### Define where to save the Red and Green raw data file based on CLA claim month:

In [6]:
base_dir = "//207.130.185.67/aqgbudget2/Cost/Reserve Adjustments/Reports/Normal Reserve Balance Verification/RVMS_Before_After_Checks"
p = Path(base_dir)
save_dir = p / CLA_MONTH
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

### Now retrieve the Excel file containing the raw "Red and Green Sheet" data:

In [31]:
df = pd.read_excel(nb.scraps['path_to_red_green_sheet_excel_file'].data)

In [32]:
df.shape

(58680, 34)

In [33]:
df.head()

,GraphCatID_After,GraphCatDesc_After,GRP_NM_After,SUBGRP_NM_After,Budgeted_CPU_SubGroup_Level_After,Planned_Sales_RVMS_After,Budgeted_CPU_GC_Level_After,Orig_Saturation_CPU_GC_Level_After,Cum_Actual_CPU_GC_Level_After,RVMS_Claim_Month_After,ModelYear_After,Factory_After,ModelName_After,DestCode_After,Group-SubGroup_After,ID_Before,GraphCatID_Before,GraphCatDesc_Before,GRP_NM_Before,SUBGRP_NM_Before,Budgeted_CPU_SubGroup_Level_Before,Planned_Sales_RVMS_Before,Budgeted_CPU_GC_Level_Before,Orig_Saturation_CPU_GC_Level_Before,Cum_Actual_CPU_GC_Level_Before,RVMS_Claim_Month_Before,ModelYear_Before,Factory_Before,ModelName_Before,DestCode_Before,Budgeted_CPU_SubGroup_Level_After_Minus_Before,Budgeted_CPU_GC_Level_After_Minus_Before,CPU_DIFF_SubGroup_Level_x_SALES,CPU_DIFF_GC_LEVEL_x_SALES
0,449,R HMI CIVIC 2010 4DR KA,Chassis,Calipers/Disk/Drums/Pads,0.83,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Calipers/Disk/Drums/Pads,"180,847.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Calipers/Disk/Drums/Pads,0.83,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.19,"3,782.32"
1,449,R HMI CIVIC 2010 4DR KA,Chassis,Drive Shaft,15.89,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Drive Shaft,"180,793.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Drive Shaft,15.89,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,28.01,"3,782.32"
2,449,R HMI CIVIC 2010 4DR KA,Chassis,Hand/cable Brakes,0.05,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Hand/cable Brakes,"180,832.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Hand/cable Brakes,0.05,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.02,"3,782.32"
3,449,R HMI CIVIC 2010 4DR KA,Chassis,Master Cylinder,0.05,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Master Cylinder,"180,790.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Master Cylinder,0.05,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.04,"3,782.32"
4,449,R HMI CIVIC 2010 4DR KA,Chassis,Steering Vibration,0.33,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Steering Vibration,"180,853.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Steering Vibration,0.33,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.01,"3,782.32"


### But we need to limit our data to just the model years that are under RVMS adjustments:

In [34]:
rvms_years = list(range(start_year, end_year + 1))

In [35]:
rvms_years

[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

In [36]:
df = df.query("ModelYear_After in(@rvms_years)")

In [37]:
df.shape

(58440, 34)

### Create exchange rate column:

In [38]:
df['Exchange_Rate'] = np.where(df['DestCode_After'] == 'KC', exchange_rate, 1)

In [39]:
df.columns

Index(['GraphCatID_After', 'GraphCatDesc_After', 'GRP_NM_After',
       'SUBGRP_NM_After', 'Budgeted_CPU_SubGroup_Level_After',
       'Planned_Sales_RVMS_After', 'Budgeted_CPU_GC_Level_After',
       'Orig_Saturation_CPU_GC_Level_After', 'Cum_Actual_CPU_GC_Level_After',
       'RVMS_Claim_Month_After', 'ModelYear_After', 'Factory_After',
       'ModelName_After', 'DestCode_After', 'Group-SubGroup_After',
       'ID_Before', 'GraphCatID_Before', 'GraphCatDesc_Before',
       'GRP_NM_Before', 'SUBGRP_NM_Before',
       'Budgeted_CPU_SubGroup_Level_Before', 'Planned_Sales_RVMS_Before',
       'Budgeted_CPU_GC_Level_Before', 'Orig_Saturation_CPU_GC_Level_Before',
       'Cum_Actual_CPU_GC_Level_Before', 'RVMS_Claim_Month_Before',
       'ModelYear_Before', 'Factory_Before', 'ModelName_Before',
       'DestCode_Before', 'Budgeted_CPU_SubGroup_Level_After_Minus_Before',
       'Budgeted_CPU_GC_Level_After_Minus_Before',
       'CPU_DIFF_SubGroup_Level_x_SALES', 'CPU_DIFF_GC_LEVEL_x_SALES',


In [40]:
df.head(10)

,GraphCatID_After,GraphCatDesc_After,GRP_NM_After,SUBGRP_NM_After,Budgeted_CPU_SubGroup_Level_After,Planned_Sales_RVMS_After,Budgeted_CPU_GC_Level_After,Orig_Saturation_CPU_GC_Level_After,Cum_Actual_CPU_GC_Level_After,RVMS_Claim_Month_After,ModelYear_After,Factory_After,ModelName_After,DestCode_After,Group-SubGroup_After,ID_Before,GraphCatID_Before,GraphCatDesc_Before,GRP_NM_Before,SUBGRP_NM_Before,Budgeted_CPU_SubGroup_Level_Before,Planned_Sales_RVMS_Before,Budgeted_CPU_GC_Level_Before,Orig_Saturation_CPU_GC_Level_Before,Cum_Actual_CPU_GC_Level_Before,RVMS_Claim_Month_Before,ModelYear_Before,Factory_Before,ModelName_Before,DestCode_Before,Budgeted_CPU_SubGroup_Level_After_Minus_Before,Budgeted_CPU_GC_Level_After_Minus_Before,CPU_DIFF_SubGroup_Level_x_SALES,CPU_DIFF_GC_LEVEL_x_SALES,Exchange_Rate
0,449,R HMI CIVIC 2010 4DR KA,Chassis,Calipers/Disk/Drums/Pads,0.83,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Calipers/Disk/Drums/Pads,"180,847.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Calipers/Disk/Drums/Pads,0.83,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.19,"3,782.32",1.00
1,449,R HMI CIVIC 2010 4DR KA,Chassis,Drive Shaft,15.89,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Drive Shaft,"180,793.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Drive Shaft,15.89,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,28.01,"3,782.32",1.00
2,449,R HMI CIVIC 2010 4DR KA,Chassis,Hand/cable Brakes,0.05,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Hand/cable Brakes,"180,832.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Hand/cable Brakes,0.05,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.02,"3,782.32",1.00
3,449,R HMI CIVIC 2010 4DR KA,Chassis,Master Cylinder,0.05,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Master Cylinder,"180,790.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Master Cylinder,0.05,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.04,"3,782.32",1.00
4,449,R HMI CIVIC 2010 4DR KA,Chassis,Steering Vibration,0.33,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Steering Vibration,"180,853.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Steering Vibration,0.33,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.01,"3,782.32",1.00
5,449,R HMI CIVIC 2010 4DR KA,Chassis,Suspension Components,0.63,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Suspension Components,"180,868.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Suspension Components,0.63,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.74,"3,782.32",1.00
6,449,R HMI CIVIC 2010 4DR KA,Chassis,Wheels,4.18,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Wheels,"180,789.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Wheels,4.18,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,4.34,"3,782.32",1.00
7,449,R HMI CIVIC 2010 4DR KA,Denso,Horn,0.31,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Denso - Horn,"180,783.00",449.00,R HMI CIVIC 2010 4DR KA,Denso,Horn,0.31,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.17,"3,782.32",1.00
8,449,R HMI CIVIC 2010 4DR KA,Denso,Power Outlet,0.05,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Denso - Power Outlet,"180,773.00",449.00,R HMI CIVIC 2010 4DR KA,Denso,Power Outlet,0.05,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.01,"3,782.32",1.00
9,449,R HMI CIVIC 2010 4DR KA,Denso,Relay/Fuse,0.51,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Denso - Relay/Fuse,"180,846.00",449.00,R HMI CIVIC 2010 4DR KA,Denso,Relay/Fuse,0.51,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.29,"3,782.32",1.00


In [8]:
# df.rename(columns={'CPU_DIFF_SubGroup_Level_x_SALES': 'Total_Adjustment_Cost_Native'}, inplace=True)

In [41]:
df.rename(columns={'CPU_DIFF_GC_LEVEL_x_SALES': 'Total_Adjustment_Cost_Native'}, inplace=True)

In [42]:
df.head(12)

,GraphCatID_After,GraphCatDesc_After,GRP_NM_After,SUBGRP_NM_After,Budgeted_CPU_SubGroup_Level_After,Planned_Sales_RVMS_After,Budgeted_CPU_GC_Level_After,Orig_Saturation_CPU_GC_Level_After,Cum_Actual_CPU_GC_Level_After,RVMS_Claim_Month_After,ModelYear_After,Factory_After,ModelName_After,DestCode_After,Group-SubGroup_After,ID_Before,GraphCatID_Before,GraphCatDesc_Before,GRP_NM_Before,SUBGRP_NM_Before,Budgeted_CPU_SubGroup_Level_Before,Planned_Sales_RVMS_Before,Budgeted_CPU_GC_Level_Before,Orig_Saturation_CPU_GC_Level_Before,Cum_Actual_CPU_GC_Level_Before,RVMS_Claim_Month_Before,ModelYear_Before,Factory_Before,ModelName_Before,DestCode_Before,Budgeted_CPU_SubGroup_Level_After_Minus_Before,Budgeted_CPU_GC_Level_After_Minus_Before,CPU_DIFF_SubGroup_Level_x_SALES,Total_Adjustment_Cost_Native,Exchange_Rate
0,449,R HMI CIVIC 2010 4DR KA,Chassis,Calipers/Disk/Drums/Pads,0.83,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Calipers/Disk/Drums/Pads,"180,847.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Calipers/Disk/Drums/Pads,0.83,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.19,"3,782.32",1.00
1,449,R HMI CIVIC 2010 4DR KA,Chassis,Drive Shaft,15.89,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Drive Shaft,"180,793.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Drive Shaft,15.89,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,28.01,"3,782.32",1.00
2,449,R HMI CIVIC 2010 4DR KA,Chassis,Hand/cable Brakes,0.05,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Hand/cable Brakes,"180,832.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Hand/cable Brakes,0.05,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.02,"3,782.32",1.00
3,449,R HMI CIVIC 2010 4DR KA,Chassis,Master Cylinder,0.05,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Master Cylinder,"180,790.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Master Cylinder,0.05,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.04,"3,782.32",1.00
4,449,R HMI CIVIC 2010 4DR KA,Chassis,Steering Vibration,0.33,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Steering Vibration,"180,853.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Steering Vibration,0.33,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.01,"3,782.32",1.00
5,449,R HMI CIVIC 2010 4DR KA,Chassis,Suspension Components,0.63,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Suspension Components,"180,868.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Suspension Components,0.63,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.74,"3,782.32",1.00
6,449,R HMI CIVIC 2010 4DR KA,Chassis,Wheels,4.18,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Wheels,"180,789.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Wheels,4.18,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,4.34,"3,782.32",1.00
7,449,R HMI CIVIC 2010 4DR KA,Denso,Horn,0.31,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Denso - Horn,"180,783.00",449.00,R HMI CIVIC 2010 4DR KA,Denso,Horn,0.31,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.17,"3,782.32",1.00
8,449,R HMI CIVIC 2010 4DR KA,Denso,Power Outlet,0.05,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Denso - Power Outlet,"180,773.00",449.00,R HMI CIVIC 2010 4DR KA,Denso,Power Outlet,0.05,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.01,"3,782.32",1.00
9,449,R HMI CIVIC 2010 4DR KA,Denso,Relay/Fuse,0.51,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Denso - Relay/Fuse,"180,846.00",449.00,R HMI CIVIC 2010 4DR KA,Denso,Relay/Fuse,0.51,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.29,"3,782.32",1.00


In [43]:
df['Total_Adjustment_Cost_USD'] = df['Total_Adjustment_Cost_Native'] * df['Exchange_Rate']
df['Budgeted_CPU_GC_Level_After_Minus_Before_USD'] = df['Budgeted_CPU_GC_Level_After_Minus_Before'] * df['Exchange_Rate']
df['Budgeted_CPU_SubGroup_Level_After_Minus_Before_USD'] = df['Budgeted_CPU_SubGroup_Level_After_Minus_Before'] * df['Exchange_Rate']
df['Budgeted_CPU_GC_Level_After_USD'] = df['Budgeted_CPU_GC_Level_After'] * df['Exchange_Rate']
df['Orig_Saturation_CPU_GC_Level_After_USD'] = df['Orig_Saturation_CPU_GC_Level_After'] * df['Exchange_Rate']
df['Cum_Actual_CPU_GC_Level_After_USD'] = df['Cum_Actual_CPU_GC_Level_After'] * df['Exchange_Rate']
df['Budgeted_CPU_GC_Level_Before_USD'] = df['Budgeted_CPU_GC_Level_Before'] * df['Exchange_Rate']

In [44]:
df.head(12)

,GraphCatID_After,GraphCatDesc_After,GRP_NM_After,SUBGRP_NM_After,Budgeted_CPU_SubGroup_Level_After,Planned_Sales_RVMS_After,Budgeted_CPU_GC_Level_After,Orig_Saturation_CPU_GC_Level_After,Cum_Actual_CPU_GC_Level_After,RVMS_Claim_Month_After,ModelYear_After,Factory_After,ModelName_After,DestCode_After,Group-SubGroup_After,ID_Before,GraphCatID_Before,GraphCatDesc_Before,GRP_NM_Before,SUBGRP_NM_Before,Budgeted_CPU_SubGroup_Level_Before,Planned_Sales_RVMS_Before,Budgeted_CPU_GC_Level_Before,Orig_Saturation_CPU_GC_Level_Before,Cum_Actual_CPU_GC_Level_Before,RVMS_Claim_Month_Before,ModelYear_Before,Factory_Before,ModelName_Before,DestCode_Before,Budgeted_CPU_SubGroup_Level_After_Minus_Before,Budgeted_CPU_GC_Level_After_Minus_Before,CPU_DIFF_SubGroup_Level_x_SALES,Total_Adjustment_Cost_Native,Exchange_Rate,Total_Adjustment_Cost_USD,Budgeted_CPU_GC_Level_After_Minus_Before_USD,Budgeted_CPU_SubGroup_Level_After_Minus_Before_USD,Budgeted_CPU_GC_Level_After_USD,Orig_Saturation_CPU_GC_Level_After_USD,Cum_Actual_CPU_GC_Level_After_USD,Budgeted_CPU_GC_Level_Before_USD
0,449,R HMI CIVIC 2010 4DR KA,Chassis,Calipers/Disk/Drums/Pads,0.83,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Calipers/Disk/Drums/Pads,"180,847.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Calipers/Disk/Drums/Pads,0.83,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.19,"3,782.32",1.00,"3,782.32",0.04,0.00,102.01,152.72,101.69,101.97
1,449,R HMI CIVIC 2010 4DR KA,Chassis,Drive Shaft,15.89,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Drive Shaft,"180,793.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Drive Shaft,15.89,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,28.01,"3,782.32",1.00,"3,782.32",0.04,0.00,102.01,152.72,101.69,101.97
2,449,R HMI CIVIC 2010 4DR KA,Chassis,Hand/cable Brakes,0.05,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Hand/cable Brakes,"180,832.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Hand/cable Brakes,0.05,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.02,"3,782.32",1.00,"3,782.32",0.04,0.00,102.01,152.72,101.69,101.97
3,449,R HMI CIVIC 2010 4DR KA,Chassis,Master Cylinder,0.05,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Master Cylinder,"180,790.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Master Cylinder,0.05,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.04,"3,782.32",1.00,"3,782.32",0.04,0.00,102.01,152.72,101.69,101.97
4,449,R HMI CIVIC 2010 4DR KA,Chassis,Steering Vibration,0.33,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Steering Vibration,"180,853.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Steering Vibration,0.33,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.01,"3,782.32",1.00,"3,782.32",0.04,0.00,102.01,152.72,101.69,101.97
5,449,R HMI CIVIC 2010 4DR KA,Chassis,Suspension Components,0.63,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Suspension Components,"180,868.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Suspension Components,0.63,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.74,"3,782.32",1.00,"3,782.32",0.04,0.00,102.01,152.72,101.69,101.97
6,449,R HMI CIVIC 2010 4DR KA,Chassis,Wheels,4.18,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Chassis - Wheels,"180,789.00",449.00,R HMI CIVIC 2010 4DR KA,Chassis,Wheels,4.18,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,4.34,"3,782.32",1.00,"3,782.32",0.04,0.00,102.01,152.72,101.69,101.97
7,449,R HMI CIVIC 2010 4DR KA,Denso,Horn,0.31,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Denso - Horn,"180,783.00",449.00,R HMI CIVIC 2010 4DR KA,Denso,Horn,0.31,"94,558.00",101.97,nan,nan,"201,812.00","2,010.00",HMI,CIVIC,KA,0.00,0.04,0.17,"3,782.32",1.00,"3,782.32",0.04,0.00,102.01,152.72,101.69,101.97
8,449,R HMI CIVIC 2010 4DR KA,Denso,Power Outlet,0.05,94558,102.01,152.72,101.69,201901,2010,HMI,CIVIC,KA,Denso 

### Create unique list of GraphCat descriptions:

In [11]:
### DEPRECATED ###
# gc_list = df[['GraphCatID_After','GraphCatDesc_After', 'Planned_Sales_RVMS_After']].drop_duplicates()

In [45]:
gc_list = df[['GraphCatID_After','GraphCatDesc_After', 'Planned_Sales_RVMS_After','Budgeted_CPU_GC_Level_After_Minus_Before_USD',
              'Budgeted_CPU_GC_Level_After_USD', 'Budgeted_CPU_GC_Level_Before_USD',
              'Orig_Saturation_CPU_GC_Level_After_USD', 'Cum_Actual_CPU_GC_Level_After_USD']].drop_duplicates()

In [46]:
gc_list.shape

(487, 8)

In [47]:
gc_list

,GraphCatID_After,GraphCatDesc_After,Planned_Sales_RVMS_After,Budgeted_CPU_GC_Level_After_Minus_Before_USD,Budgeted_CPU_GC_Level_After_USD,Budgeted_CPU_GC_Level_Before_USD,Orig_Saturation_CPU_GC_Level_After_USD,Cum_Actual_CPU_GC_Level_After_USD
0,449,R HMI CIVIC 2010 4DR KA,94558,0.04,102.01,101.97,152.72,101.69
120,450,R MAP Accord 2010 L4 KA,225412,0.45,105.92,105.47,177.24,102.60
240,470,R MAP Accord 2010 V6 KA,44941,0.00,229.81,229.81,280.50,229.51
360,484,R ELP CRV 2010 4WD KA,102615,0.00,92.30,92.30,127.33,92.39
480,485,R ELP CRV 2010 2WD KA,51299,0.00,75.27,75.27,119.83,74.27
600,490,R MAP Accord 2010 L4 KC,9840,0.00,193.53,193.53,183.09,193.53
720,492,R MAP Accord 2010 V6 KC,2100,0.00,281.38,281.38,273.78,279.87
840,493,R ELP Element 2010 2WD KA,7080,0.00,91.25,91.25,169.87,89.64
960,494,R ELP Element 2010 2WD KC,60,0.00,147.07,147.07,153.50,146.72
1080,495,R ELP Element 2010 4WD KA,9480,0.00,91.10,91.10,175.84,89.50


### Create helper functions to Add Model Year, Factory, and Model Name columns:

In [48]:
def getModelYear(row) -> str:
    # executing strip() also because someone can make a graphcat description with a trailing whitespace
    word_token = row['GraphCatDesc_After'].strip().split()
    
    model_year = word_token[3]
    
    if model_year.isdigit():
        return model_year
    else:
        return word_token[4]

def getFactoryCode(row) -> str:
    # executing strip() also because someone can make a graphcat description with a trailing whitespace
    word_token = row['GraphCatDesc_After'].strip().split()
    factory_code = word_token[1]
    
    return factory_code.upper()

def getModelName(row) -> str:
    # executing strip() also because someone can make a graphcat description with a trailing whitespace
    word_token = row['GraphCatDesc_After'].strip().split()
    model_name = word_token[2]
    
    return model_name.upper()

def getDestCode(row) -> str:
    # executing strip() also because someone can make a graphcat description with a trailing whitespace
    word_token = row['GraphCatDesc_After'].strip().split()
    destination_code = word_token[-1]
    
    return destination_code.upper()

### Apply the above functions to create the model year, factory, and model name columns

In [49]:
gc_list['ModelYear'] = gc_list.apply(getModelYear, axis='columns')
gc_list['Factory'] = gc_list.apply(getFactoryCode, axis='columns')
gc_list['ModelName'] = gc_list.apply(getModelName, axis='columns')
gc_list['DestCodeCustom'] = gc_list.apply(getDestCode, axis='columns')

#### Let's confirm the new columns were added:

In [50]:
gc_list.head()

,GraphCatID_After,GraphCatDesc_After,Planned_Sales_RVMS_After,Budgeted_CPU_GC_Level_After_Minus_Before_USD,Budgeted_CPU_GC_Level_After_USD,Budgeted_CPU_GC_Level_Before_USD,Orig_Saturation_CPU_GC_Level_After_USD,Cum_Actual_CPU_GC_Level_After_USD,ModelYear,Factory,ModelName,DestCodeCustom
0,449,R HMI CIVIC 2010 4DR KA,94558,0.04,102.01,101.97,152.72,101.69,2010,HMI,CIVIC,KA
120,450,R MAP Accord 2010 L4 KA,225412,0.45,105.92,105.47,177.24,102.60,2010,MAP,ACCORD,KA
240,470,R MAP Accord 2010 V6 KA,44941,0.00,229.81,229.81,280.50,229.51,2010,MAP,ACCORD,KA
360,484,R ELP CRV 2010 4WD KA,102615,0.00,92.30,92.30,127.33,92.39,2010,ELP,CRV,KA
480,485,R ELP CRV 2010 2WD KA,51299,0.00,75.27,75.27,119.83,74.27,2010,ELP,CRV,KA


### DEPRECATED Create pivot table where ```graphcat description``` is rows and sum of ```CPU_DIFF_SubGroup_Level_x_SALES``` column:

Basically, this is our list of top total cost adjustments by GraphCat description.

### Create Unique list of GraphCats and their CPUs at the GraphCat level

Basically, this is our list of top total cost adjustments by GraphCat description.

In [52]:
total_adj_gc_level = df[['GraphCatDesc_After', 'Total_Adjustment_Cost_USD']].drop_duplicates()

In [53]:
total_adj_gc_level

,GraphCatDesc_After,Total_Adjustment_Cost_USD
0,R HMI CIVIC 2010 4DR KA,"3,782.32"
120,R MAP Accord 2010 L4 KA,"101,435.40"
240,R MAP Accord 2010 V6 KA,0.00
360,R ELP CRV 2010 4WD KA,0.00
480,R ELP CRV 2010 2WD KA,0.00
600,R MAP Accord 2010 L4 KC,0.00
720,R MAP Accord 2010 V6 KC,0.00
840,R ELP Element 2010 2WD KA,0.00
960,R ELP Element 2010 2WD KC,0.00
1080,R ELP Element 2010 4WD KA,0.00


### Create list of CPU differences at the GraphCat level:

In [20]:
### DEPRECATED ###
# cpu_diff_gc_level = df[['GraphCatDesc_After', 'Budgeted_CPU_GC_Level_After_Minus_Before_USD']].drop_duplicates()

In [21]:
# cpu_diff_gc_level

### Create pivot table where GraphCat-SubGroup is the rows and sum the SubGroup level graphcat CPU adjustments:

In [54]:
pivot = df.pivot_table(values=['Budgeted_CPU_SubGroup_Level_After_Minus_Before_USD'], index=['GraphCatDesc_After','Group-SubGroup_After'], aggfunc='sum')

In [55]:
pivot

Budgeted_CPU_SubGroup_Level_After_Minus_Before_USD
GraphCatDesc_After         Group-SubGroup_After                                                                             
R ELP CROSSTOUR 2014 L4 KA Chassis - ABS/TCS/VSA/BSC                                                                   3.22 
                           Chassis - Alignment/Drift/SWOC                                                              0.00 
                           Chassis - Brake Judder                                                                      1.35 
                           Chassis - Brake Noise                                                                      -0.00 
                           Chassis - Brake Pipes/hoses                                                                 0.00 
                           Chassis - Calipers/Disk/Drums/Pads                                                         -0.00 
                           Chassis - Clutch                                                                            0.00 
                           Chassis - Conventional Brakes (pedal/booster)                                              -0.00 
                           Chassis - Dampers                                                                           2.41 
                           Chassis - Diff, Transf Controls                                                             0.00 
                           Chassis - Drive Shaft                                                                       6.68 
                           Chassis - Engine Trim                                                                       0.00 
                           Chassis - Gear Shifter/Select Lever                                                        -0.00 
                           Chassis - Hand/cable Brakes                                                                 0.00 
                           Chassis - Jack                                                                              0.00 
                           Chassis - Master Cylinder                                                                   0.00 
                           Chassis - Mounts - Engine/mission                                                           0.56 
                           Chassis - P/S Gearbox                                                                      -0.05 
                           Chassis - P/S Pump/hoses                                                                   -0.02 
                           Chassis - Pedals                                                                            0.00 
                           Chassis - Silencer/Exhaust Pipes                                                            0.00 
                           Chassis - Steering System                                                                   1.72 
                           Chassis - Steering Vibration                                                               -0.09 
                           Chassis - Suspension Components                                                             1.72 
                           Chassis - Wheel Bearings                                                                   -0.33 
                           Chassis - Wheels                                                                           39.26 
                           Denso - Audio                                                                              -0.13 
                           Denso - Battery                                                                             0.01 
                           Denso - Bulbs                                                                               0.48 
                           Denso - Center Module                                                                      -0.00 
                           Denso - Combimeter                             

### But...how do we obtain the top 3 and bottom 3 adjustments at the subgroup level??!!  Google search to the rescue!!!

### Found this StackOverflow [example](https://stackoverflow.com/questions/45365923/how-to-use-nlargest-on-multilevel-pivot-table-in-pandas)

In [56]:
top3 = pivot.groupby(level='GraphCatDesc_After')['Budgeted_CPU_SubGroup_Level_After_Minus_Before_USD'].nlargest(3).reset_index(level=0, drop=True).reset_index()
bottom3 = pivot.groupby(level='GraphCatDesc_After')['Budgeted_CPU_SubGroup_Level_After_Minus_Before_USD'].nsmallest(3).reset_index(level=0, drop=True).reset_index()

### Now merge or concatenate the 2 data sets together along the row axis direction:

In [57]:
top3_bottom3 = pd.concat([top3, bottom3], axis='rows')

In [58]:
top3_bottom3.head(12)

,GraphCatDesc_After,Group-SubGroup_After,Budgeted_CPU_SubGroup_Level_After_Minus_Before_USD
0,R ELP CROSSTOUR 2014 L4 KA,Chassis - Wheels,39.26
1,R ELP CROSSTOUR 2014 L4 KA,Exterior - Front Light Housing,13.55
2,R ELP CROSSTOUR 2014 L4 KA,Denso - Navi,9.78
3,R ELP CROSSTOUR 2014 L4 KC,Chassis - Engine Trim,0.00
4,R ELP CROSSTOUR 2014 L4 KC,Engine - CVT Trans,0.00
5,R ELP CROSSTOUR 2014 L4 KC,Transmission - Seals,0.00
6,R ELP CROSSTOUR 2014 V6 KA,Engine - Exhaust Manifold,0.70
7,R ELP CROSSTOUR 2014 V6 KA,Transmission - MIL On,0.55
8,R ELP CROSSTOUR 2014 V6 KA,Engine - Oil Leaks,0.42
9,R ELP CROSSTOUR 2014 V6 KC,Engine - Timing Components,1.98


#### Sort by GraphCat and SubGroup CPU column in descending order:

In [59]:
top3_bottom3.sort_values(by=['GraphCatDesc_After','Budgeted_CPU_SubGroup_Level_After_Minus_Before_USD'], ascending=[False, False], inplace=True)

In [60]:
top3_bottom3.head(12)

,GraphCatDesc_After,Group-SubGroup_After,Budgeted_CPU_SubGroup_Level_After_Minus_Before_USD
1458,R PMC NSX 2019 KC,Chassis - ABS/TCS/VSA/BSC,0.00
1459,R PMC NSX 2019 KC,Chassis - Alignment/Drift/SWOC,0.00
1460,R PMC NSX 2019 KC,Chassis - Brake Judder,0.00
1458,R PMC NSX 2019 KC,Chassis - ABS/TCS/VSA/BSC,0.00
1459,R PMC NSX 2019 KC,Chassis - Alignment/Drift/SWOC,0.00
1460,R PMC NSX 2019 KC,Chassis - Brake Judder,0.00
1455,R PMC NSX 2019 KA,Chassis - ABS/TCS/VSA/BSC,0.00
1456,R PMC NSX 2019 KA,Chassis - Alignment/Drift/SWOC,0.00
1457,R PMC NSX 2019 KA,Chassis - Brake Judder,0.00
1455,R PMC NSX 2019 KA,Chassis - ABS/TCS/VSA/BSC,0.00


**From above, we can see that for each GraphCat, we have the top 3 subgroup CPU adjustment and bottom 3 subgroup adjustment!**

### Merge with the previously created data sets to obtain additional columns:

In [29]:
### DEPRECATED ###
# top3_bottom3 = pd.merge(top3_bottom3, total_adj_gc_level, how='left', left_on=['GraphCatDesc_After'], right_index=True)
# top3_bottom3 = pd.merge(top3_bottom3, cpu_diff_gc_level, how='left', left_on=['GraphCatDesc_After'], right_on=['GraphCatDesc_After'])
# top3_bottom3 = pd.merge(top3_bottom3, gc_list, how='left', left_on=['GraphCatDesc_After'], right_on=['GraphCatDesc_After'])

In [61]:
top3_bottom3 = pd.merge(top3_bottom3, total_adj_gc_level, how='left', left_on=['GraphCatDesc_After'], right_on=['GraphCatDesc_After'])
top3_bottom3 = pd.merge(top3_bottom3, gc_list, how='left', left_on=['GraphCatDesc_After'], right_on=['GraphCatDesc_After'])

In [62]:
top3_bottom3.head(12)

,GraphCatDesc_After,Group-SubGroup_After,Budgeted_CPU_SubGroup_Level_After_Minus_Before_USD,Total_Adjustment_Cost_USD,GraphCatID_After,Planned_Sales_RVMS_After,Budgeted_CPU_GC_Level_After_Minus_Before_USD,Budgeted_CPU_GC_Level_After_USD,Budgeted_CPU_GC_Level_Before_USD,Orig_Saturation_CPU_GC_Level_After_USD,Cum_Actual_CPU_GC_Level_After_USD,ModelYear,Factory,ModelName,DestCodeCustom
0,R PMC NSX 2019 KC,Chassis - ABS/TCS/VSA/BSC,0.00,nan,1742,39,nan,"3,561.79",nan,"3,561.79",nan,2019,PMC,NSX,KC
1,R PMC NSX 2019 KC,Chassis - Alignment/Drift/SWOC,0.00,nan,1742,39,nan,"3,561.79",nan,"3,561.79",nan,2019,PMC,NSX,KC
2,R PMC NSX 2019 KC,Chassis - Brake Judder,0.00,nan,1742,39,nan,"3,561.79",nan,"3,561.79",nan,2019,PMC,NSX,KC
3,R PMC NSX 2019 KC,Chassis - ABS/TCS/VSA/BSC,0.00,nan,1742,39,nan,"3,561.79",nan,"3,561.79",nan,2019,PMC,NSX,KC
4,R PMC NSX 2019 KC,Chassis - Alignment/Drift/SWOC,0.00,nan,1742,39,nan,"3,561.79",nan,"3,561.79",nan,2019,PMC,NSX,KC
5,R PMC NSX 2019 KC,Chassis - Brake Judder,0.00,nan,1742,39,nan,"3,561.79",nan,"3,561.79",nan,2019,PMC,NSX,KC
6,R PMC NSX 2019 KA,Chassis - ABS/TCS/VSA/BSC,0.00,nan,1740,291,nan,"3,634.82",nan,"3,634.82",82.73,2019,PMC,NSX,KA
7,R PMC NSX 2019 KA,Chassis - Alignment/Drift/SWOC,0.00,nan,1740,291,nan,"3,634.82",nan,"3,634.82",82.73,2019,PMC,NSX,KA
8,R PMC NSX 2019 KA,Chassis - Brake Judder,0.00,nan,1740,291,nan,"3,634.82",nan,"3,634.82",82.73,2019,PMC,NSX,KA
9,R PMC NSX 2019 KA,Chassis - ABS/TCS/VSA/BSC,0.00,nan,1740,291,nan,"3,634.82",nan,"3,634.82",82.73,2019,PMC,NSX,KA


### Confirm our data set is sorted by GraphCat total adjustment amount, then GraphCat, and then SubGrpu CPU amount:

In [63]:
top3_bottom3.columns

Index(['GraphCatDesc_After', 'Group-SubGroup_After',
       'Budgeted_CPU_SubGroup_Level_After_Minus_Before_USD',
       'Total_Adjustment_Cost_USD', 'GraphCatID_After',
       'Planned_Sales_RVMS_After',
       'Budgeted_CPU_GC_Level_After_Minus_Before_USD',
       'Budgeted_CPU_GC_Level_After_USD', 'Budgeted_CPU_GC_Level_Before_USD',
       'Orig_Saturation_CPU_GC_Level_After_USD',
       'Cum_Actual_CPU_GC_Level_After_USD', 'ModelYear', 'Factory',
       'ModelName', 'DestCodeCustom'],
      dtype='object')

In [64]:
top3_bottom3.sort_values(by=['Total_Adjustment_Cost_USD','GraphCatDesc_After','Budgeted_CPU_SubGroup_Level_After_Minus_Before_USD'], 
                                        ascending=[False, False, False], inplace=True)

In [65]:
top3_bottom3.head(12)

,GraphCatDesc_After,Group-SubGroup_After,Budgeted_CPU_SubGroup_Level_After_Minus_Before_USD,Total_Adjustment_Cost_USD,GraphCatID_After,Planned_Sales_RVMS_After,Budgeted_CPU_GC_Level_After_Minus_Before_USD,Budgeted_CPU_GC_Level_After_USD,Budgeted_CPU_GC_Level_Before_USD,Orig_Saturation_CPU_GC_Level_After_USD,Cum_Actual_CPU_GC_Level_After_USD,ModelYear,Factory,ModelName,DestCodeCustom
1560,R HDM CRV 2014 KX,Chassis - Steering System,22.69,"4,966,219.58",1376,28763,172.66,262.65,89.99,262.65,239.90,2014,HDM,CRV,KX
1561,R HDM CRV 2014 KX,Engine - Oil Leaks,14.22,"4,966,219.58",1376,28763,172.66,262.65,89.99,262.65,239.90,2014,HDM,CRV,KX
1562,R HDM CRV 2014 KX,Denso - Battery,12.99,"4,966,219.58",1376,28763,172.66,262.65,89.99,262.65,239.90,2014,HDM,CRV,KX
1563,R HDM CRV 2014 KX,Chassis - Brake Pipes/hoses,0.00,"4,966,219.58",1376,28763,172.66,262.65,89.99,262.65,239.90,2014,HDM,CRV,KX
1564,R HDM CRV 2014 KX,Chassis - Clutch,0.00,"4,966,219.58",1376,28763,172.66,262.65,89.99,262.65,239.90,2014,HDM,CRV,KX
1565,R HDM CRV 2014 KX,Chassis - Engine Trim,0.00,"4,966,219.58",1376,28763,172.66,262.65,89.99,262.65,239.90,2014,HDM,CRV,KX
1134,R HMA PILOT 2016 6AT KA,Engine - Fuel System,54.38,"4,862,961.01",1429,143747,33.83,320.58,286.75,206.20,239.66,2016,HMA,PILOT,KA
1135,R HMA PILOT 2016 6AT KA,OBD - ECU/PCM/TCM,21.95,"4,862,961.01",1429,143747,33.83,320.58,286.75,206.20,239.66,2016,HMA,PILOT,KA
1136,R HMA PILOT 2016 6AT KA,OBD - Missfire,1.86,"4,862,961.01",1429,143747,33.83,320.58,286.75,206.20,239.66,2016,HMA,PILOT,KA
1137,R HMA PILOT 2016 6AT KA,OBD - O2 Sensor,-0.15,"4,862,961.01",1429,143747,33.83,320.58,286.75,206.20,239.66,2016,HMA,PILOT,KA


#### We need a way to "blank out" / "zero out" repeating values in the ```CPU_DIFF_SubGroup_Level_x_SALES``` column and ```Budgeted_CPU_GC_Level_After_Minus_Before``` column.  But how?!

### SOLUTION: Create "ROW_NUM" column and then identify rows using the ROW_NUM value.

In [66]:
top3_bottom3['ROW_NUM'] = top3_bottom3.groupby(['GraphCatDesc_After']).cumcount() + 1

In [67]:
top3_bottom3.head(6)

,GraphCatDesc_After,Group-SubGroup_After,Budgeted_CPU_SubGroup_Level_After_Minus_Before_USD,Total_Adjustment_Cost_USD,GraphCatID_After,Planned_Sales_RVMS_After,Budgeted_CPU_GC_Level_After_Minus_Before_USD,Budgeted_CPU_GC_Level_After_USD,Budgeted_CPU_GC_Level_Before_USD,Orig_Saturation_CPU_GC_Level_After_USD,Cum_Actual_CPU_GC_Level_After_USD,ModelYear,Factory,ModelName,DestCodeCustom,ROW_NUM
1560,R HDM CRV 2014 KX,Chassis - Steering System,22.69,"4,966,219.58",1376,28763,172.66,262.65,89.99,262.65,239.90,2014,HDM,CRV,KX,1
1561,R HDM CRV 2014 KX,Engine - Oil Leaks,14.22,"4,966,219.58",1376,28763,172.66,262.65,89.99,262.65,239.90,2014,HDM,CRV,KX,2
1562,R HDM CRV 2014 KX,Denso - Battery,12.99,"4,966,219.58",1376,28763,172.66,262.65,89.99,262.65,239.90,2014,HDM,CRV,KX,3
1563,R HDM CRV 2014 KX,Chassis - Brake Pipes/hoses,0.00,"4,966,219.58",1376,28763,172.66,262.65,89.99,262.65,239.90,2014,HDM,CRV,KX,4
1564,R HDM CRV 2014 KX,Chassis - Clutch,0.00,"4,966,219.58",1376,28763,172.66,262.65,89.99,262.65,239.90,2014,HDM,CRV,KX,5
1565,R HDM CRV 2014 KX,Chassis - Engine Trim,0.00,"4,966,219.58",1376,28763,172.66,262.65,89.99,262.65,239.90,2014,HDM,CRV,KX,6


### Perform IF-ELSE logic to "blank out" / "zero out" repeating values in the 2 columns:

In [68]:
# If ROW_NUM == 1, then keep the orginal value, otherwise, make it zero/0
top3_bottom3['Total_Adjustment_Cost_USD'] = np.where(top3_bottom3['ROW_NUM'] == 1, 
                                                                          top3_bottom3['Total_Adjustment_Cost_USD'], 0)
top3_bottom3['Budgeted_CPU_GC_Level_After_Minus_Before_USD'] = np.where(top3_bottom3['ROW_NUM'] == 1, 
                                                                          top3_bottom3['Budgeted_CPU_GC_Level_After_Minus_Before_USD'], 0)
top3_bottom3['Budgeted_CPU_GC_Level_After_USD'] = np.where(top3_bottom3['ROW_NUM'] == 1, 
                                                                          top3_bottom3['Budgeted_CPU_GC_Level_After_USD'], 0)
top3_bottom3['Budgeted_CPU_GC_Level_Before_USD'] = np.where(top3_bottom3['ROW_NUM'] == 1, 
                                                                          top3_bottom3['Budgeted_CPU_GC_Level_Before_USD'], 0)
top3_bottom3['Orig_Saturation_CPU_GC_Level_After_USD'] = np.where(top3_bottom3['ROW_NUM'] == 1, 
                                                                          top3_bottom3['Orig_Saturation_CPU_GC_Level_After_USD'], 0)
top3_bottom3['Cum_Actual_CPU_GC_Level_After_USD'] = np.where(top3_bottom3['ROW_NUM'] == 1, 
                                                                          top3_bottom3['Cum_Actual_CPU_GC_Level_After_USD'], 0)

### Let's see if that worked:

In [69]:
top3_bottom3.head(6)

,GraphCatDesc_After,Group-SubGroup_After,Budgeted_CPU_SubGroup_Level_After_Minus_Before_USD,Total_Adjustment_Cost_USD,GraphCatID_After,Planned_Sales_RVMS_After,Budgeted_CPU_GC_Level_After_Minus_Before_USD,Budgeted_CPU_GC_Level_After_USD,Budgeted_CPU_GC_Level_Before_USD,Orig_Saturation_CPU_GC_Level_After_USD,Cum_Actual_CPU_GC_Level_After_USD,ModelYear,Factory,ModelName,DestCodeCustom,ROW_NUM
1560,R HDM CRV 2014 KX,Chassis - Steering System,22.69,"4,966,219.58",1376,28763,172.66,262.65,89.99,262.65,239.90,2014,HDM,CRV,KX,1
1561,R HDM CRV 2014 KX,Engine - Oil Leaks,14.22,0.00,1376,28763,0.00,0.00,0.00,0.00,0.00,2014,HDM,CRV,KX,2
1562,R HDM CRV 2014 KX,Denso - Battery,12.99,0.00,1376,28763,0.00,0.00,0.00,0.00,0.00,2014,HDM,CRV,KX,3
1563,R HDM CRV 2014 KX,Chassis - Brake Pipes/hoses,0.00,0.00,1376,28763,0.00,0.00,0.00,0.00,0.00,2014,HDM,CRV,KX,4
1564,R HDM CRV 2014 KX,Chassis - Clutch,0.00,0.00,1376,28763,0.00,0.00,0.00,0.00,0.00,2014,HDM,CRV,KX,5
1565,R HDM CRV 2014 KX,Chassis - Engine Trim,0.00,0.00,1376,28763,0.00,0.00,0.00,0.00,0.00,2014,HDM,CRV,KX,6


#### Nice, it worked!

### Rename columns by creating a Python dictionary data structure:

In [70]:
rename_columns_mapper = {'GraphCatDesc_After': 'GraphCatDesc', 
                         'Group-SubGroup_After': 'Group-SubGroup',
                         'Budgeted_CPU_SubGroup_Level_After_Minus_Before_USD': 'Total_CPU_Adj_at_SubGroup_Level_USD',
                         'Budgeted_CPU_GC_Level_After_Minus_Before_USD': 'Total_CPU_Adj_at_GraphCat_Level_USD',
                         'GraphCatID_After': 'GraphCatID',
                         'Planned_Sales_RVMS_After': 'Planned_Sales',
                         'Orig_Saturation_CPU_GC_Level_After_USD': 'Orig_Saturation_CPU_GC_Level_USD',
                         'Cum_Actual_CPU_GC_Level_After_USD': 'Cum_Actual_CPU_GC_Level_USD'
                        }

#### Then apply pandas' ```rename()``` function:

In [71]:
top3_bottom3.rename(rename_columns_mapper, axis='columns', inplace=True)

In [72]:
top3_bottom3.head()

,GraphCatDesc,Group-SubGroup,Total_CPU_Adj_at_SubGroup_Level_USD,Total_Adjustment_Cost_USD,GraphCatID,Planned_Sales,Total_CPU_Adj_at_GraphCat_Level_USD,Budgeted_CPU_GC_Level_After_USD,Budgeted_CPU_GC_Level_Before_USD,Orig_Saturation_CPU_GC_Level_USD,Cum_Actual_CPU_GC_Level_USD,ModelYear,Factory,ModelName,DestCodeCustom,ROW_NUM
1560,R HDM CRV 2014 KX,Chassis - Steering System,22.69,"4,966,219.58",1376,28763,172.66,262.65,89.99,262.65,239.90,2014,HDM,CRV,KX,1
1561,R HDM CRV 2014 KX,Engine - Oil Leaks,14.22,0.00,1376,28763,0.00,0.00,0.00,0.00,0.00,2014,HDM,CRV,KX,2
1562,R HDM CRV 2014 KX,Denso - Battery,12.99,0.00,1376,28763,0.00,0.00,0.00,0.00,0.00,2014,HDM,CRV,KX,3
1563,R HDM CRV 2014 KX,Chassis - Brake Pipes/hoses,0.00,0.00,1376,28763,0.00,0.00,0.00,0.00,0.00,2014,HDM,CRV,KX,4
1564,R HDM CRV 2014 KX,Chassis - Clutch,0.00,0.00,1376,28763,0.00,0.00,0.00,0.00,0.00,2014,HDM,CRV,KX,5


### I want to now re-order columns

#### Let's get print out of column names:

In [73]:
top3_bottom3.columns

Index(['GraphCatDesc', 'Group-SubGroup', 'Total_CPU_Adj_at_SubGroup_Level_USD',
       'Total_Adjustment_Cost_USD', 'GraphCatID', 'Planned_Sales',
       'Total_CPU_Adj_at_GraphCat_Level_USD',
       'Budgeted_CPU_GC_Level_After_USD', 'Budgeted_CPU_GC_Level_Before_USD',
       'Orig_Saturation_CPU_GC_Level_USD', 'Cum_Actual_CPU_GC_Level_USD',
       'ModelYear', 'Factory', 'ModelName', 'DestCodeCustom', 'ROW_NUM'],
      dtype='object')

#### Now, re-order the column names:

In [74]:
top3_bottom3 = top3_bottom3[['GraphCatID',
                             'GraphCatDesc',
                             'ModelYear',
                             'Factory',
                             'ModelName',
                             'DestCodeCustom',
                             'Total_Adjustment_Cost_USD',
                             'Total_CPU_Adj_at_GraphCat_Level_USD',
                             'Budgeted_CPU_GC_Level_After_USD',
                             'Budgeted_CPU_GC_Level_Before_USD',
                             'Group-SubGroup',
                             'Total_CPU_Adj_at_SubGroup_Level_USD',
                             'ROW_NUM',
                             'Planned_Sales',
                             'Orig_Saturation_CPU_GC_Level_USD',
                             'Cum_Actual_CPU_GC_Level_USD'
                            ]]

In [75]:
top3_bottom3.head(12)

,GraphCatID,GraphCatDesc,ModelYear,Factory,ModelName,DestCodeCustom,Total_Adjustment_Cost_USD,Total_CPU_Adj_at_GraphCat_Level_USD,Budgeted_CPU_GC_Level_After_USD,Budgeted_CPU_GC_Level_Before_USD,Group-SubGroup,Total_CPU_Adj_at_SubGroup_Level_USD,ROW_NUM,Planned_Sales,Orig_Saturation_CPU_GC_Level_USD,Cum_Actual_CPU_GC_Level_USD
1560,1376,R HDM CRV 2014 KX,2014,HDM,CRV,KX,"4,966,219.58",172.66,262.65,89.99,Chassis - Steering System,22.69,1,28763,262.65,239.90
1561,1376,R HDM CRV 2014 KX,2014,HDM,CRV,KX,0.00,0.00,0.00,0.00,Engine - Oil Leaks,14.22,2,28763,0.00,0.00
1562,1376,R HDM CRV 2014 KX,2014,HDM,CRV,KX,0.00,0.00,0.00,0.00,Denso - Battery,12.99,3,28763,0.00,0.00
1563,1376,R HDM CRV 2014 KX,2014,HDM,CRV,KX,0.00,0.00,0.00,0.00,Chassis - Brake Pipes/hoses,0.00,4,28763,0.00,0.00
1564,1376,R HDM CRV 2014 KX,2014,HDM,CRV,KX,0.00,0.00,0.00,0.00,Chassis - Clutch,0.00,5,28763,0.00,0.00
1565,1376,R HDM CRV 2014 KX,2014,HDM,CRV,KX,0.00,0.00,0.00,0.00,Chassis - Engine Trim,0.00,6,28763,0.00,0.00
1134,1429,R HMA PILOT 2016 6AT KA,2016,HMA,PILOT,KA,"4,862,961.01",33.83,320.58,286.75,Engine - Fuel System,54.38,1,143747,206.20,239.66
1135,1429,R HMA PILOT 2016 6AT KA,2016,HMA,PILOT,KA,0.00,0.00,0.00,0.00,OBD - ECU/PCM/TCM,21.95,2,143747,0.00,0.00
1136,1429,R HMA PILOT 2016 6AT KA,2016,HMA,PILOT,KA,0.00,0.00,0.00,0.00,OBD - Missfire,1.86,3,143747,0.00,0.00
1137,1429,R HMA PILOT 2016 6AT KA,2016,HMA,PILOT,KA,0.00,0.00,0.00,0.00,OBD - O2 Sensor,-0.15,4,143747,0.00,0.00


### We're done!  Now we can export to Excel, to clipboard, etc

In [45]:
# top3_bottom3.to_excel(r'D:\temp\top3_bottom3_USD.xlsx', index=False)

In [ ]:
file_name = 'top3_bottom3_USD.xlsx'

In [ ]:
top3_bottom3.to_excel(save_dir / file_name, index=False)

### If script made it this far, send out Windows 10 toast notification:

In [ ]:
toaster = ToastNotifier()
toaster.show_toast("### Job Status ###",
                   "Successfuly Summarized Red and Green Sheet Data in USD",
                   icon_path="images/honda_logo.ico",
                   duration=5)